In [14]:
import pandas as pd
import numpy as np
import datetime

In [2]:
auctions = pd.read_csv("auctions.csv", dtype={"ref_type_id": np.uint8, "source_id": np.uint8})

In [3]:
auctions["date"] =  pd.to_datetime(auctions["date"], errors = "coerce")

In [34]:
def create_window(df, initial_day, final_day, date_feature_name, id_feature_name, label_name):
    window = df.loc[(df[date_feature_name].dt.day >= initial_day) & (df[date_feature_name].dt.day <= final_day)]
    window = window.groupby(id_feature_name).agg({date_feature_name : 'min'}).reset_index()
    window[label_name] = (window[date_feature_name] - pd.to_datetime("2019-04-" + str(initial_day))).dt.total_seconds()
    return window

In [37]:
window_18to20 = create_window(auctions, 18, 20, "date", "device_id", "seconds_to_appear")

In [38]:
window_18to20.head()

,device_id,date,seconds_to_appear
0,41863526108385,2019-04-19 19:40:28.465866,157228.465866
1,135153013040192,2019-04-20 04:10:54.009137,187854.009137
2,161514654074162,2019-04-18 02:52:46.357746,10366.357746
3,181891380775191,2019-04-20 23:19:25.420614,256765.420614
4,186034136943920,2019-04-18 16:42:46.331894,60166.331894


## Ignorar esto (es un feature)

In [10]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].fillna(259200) #to check
    df_sorted = df_sorted.groupby('device_id').agg({'time_to_reappear': 'mean'})
    return df_sorted

# Pruebas

In [ ]:
"""X, y = window_18to20.iloc[:,:-2], window_18to20[["seconds_to_appear"]] #X sera el dataframe con todos los features
#y es el label (en X DEVICE ID NO VAN!!)
#window_18to20.iloc[:,:-1] en este caso no me quedo con la serie date porque dmatrix solo acepta valores numericos """

In [ ]:
df_has_installs = pd.DataFrame(installs['ref_hash'].unique())
df_has_installs['has_installs'] = 1
df_has_installs.columns = ['ref_hash', 'has_installs']

In [ ]:
auctions_features = add_feature(auctions_features, df_has_installs, 'ref_hash', 'ref_hash', 'has_installs', 0)
auctions_features.head()

In [ ]:
X = X[['has_installs']]
X.head()

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label= y) #en realidad aca va el label de la siguiente ventana !!!! 

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg.fit(X,y) #X es la data de train e y es el label de train

In [ ]:
window_21to23 = create_window(auctions, 21, 23, "date", "device_id", "seconds_to_appear")

In [ ]:
X_test, X_real_label = window_21to23.iloc[:,:-2], window_21to23.iloc[:,-1]
X_test = add_feature(X_test, df_has_installs, 'device_id', 'ref_hash', 'has_installs', 0)
X_test = X_test[['has_installs']]
predictions = xg_reg.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(X_real_label, predictions))
print("RMSE: %f" % (rmse))